In [1]:
#@title set up mount symlink
import os
home = os.path.expanduser('~')

In [2]:
##### @title download mujoco

MJC_PATH = '{}/.mujoco'.format(home)
if not os.path.exists(MJC_PATH):
    %mkdir $MJC_PATH
%cd $MJC_PATH
if not os.path.exists(os.path.join(MJC_PATH, 'mujoco200')):
    !wget -q https://www.roboti.us/download/mujoco200_linux.zip
    !unzip -q mujoco200_linux.zip
    %mv mujoco200_linux mujoco200
    %rm mujoco200_linux.zip

/Users/ikasou/Downloads


In [4]:
#@title update mujoco paths

ex_path = home + '/Documents/Python/BerkeleyDRL_fall2021/hw2'
MJC_PATH = home + '/.mujoco'
if home[:6] == '/Users':
    os.environ['LIBRARY_PATH'] += '/usr/local/lib'
    os.environ['LIBRARY_PATH'] += ':{}/mujoco200/bin'.format(MJC_PATH)   
else:
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/nvidia/lib64'
    os.environ['LD_LIBRARY_PATH'] += ':{}/mujoco200/bin'.format(MJC_PATH)

os.environ['MUJOCO_PY_MUJOCO_PATH'] = '{}/mujoco200'.format(MJC_PATH)
os.environ['MUJOCO_PY_MJKEY_PATH'] = '{}/mjkey.txt'.format(MJC_PATH)

Ensure your `mjkey.txt` is in /content/cs285_f2021/mujoco before this step

In [5]:
#@title clone and install mujoco-py
'''
%cd $MJC_PATH
if not os.path.exists('mujoco-py'):
  !git clone https://github.com/openai/mujoco-py.git
%cd mujoco-py
%pip install -e .
'''
## cythonize at the first import
import mujoco_py

objc[3243]: Class GLFWApplicationDelegate is implemented in both /Users/ikasou/Downloads/mujoco200/bin/libglfw.3.dylib (0x104f71778) and /Users/ikasou/Library/Python/3.8/lib/python/site-packages/glfw/libglfw.3.dylib (0x1050e06e8). One of the two will be used. Which one is undefined.
objc[3243]: Class GLFWWindowDelegate is implemented in both /Users/ikasou/Downloads/mujoco200/bin/libglfw.3.dylib (0x104f71700) and /Users/ikasou/Library/Python/3.8/lib/python/site-packages/glfw/libglfw.3.dylib (0x1050e0710). One of the two will be used. Which one is undefined.
objc[3243]: Class GLFWContentView is implemented in both /Users/ikasou/Downloads/mujoco200/bin/libglfw.3.dylib (0x104f717a0) and /Users/ikasou/Library/Python/3.8/lib/python/site-packages/glfw/libglfw.3.dylib (0x1050e0760). One of the two will be used. Which one is undefined.
objc[3243]: Class GLFWWindow is implemented in both /Users/ikasou/Downloads/mujoco200/bin/libglfw.3.dylib (0x104f71818) and /Users/ikasou/Library/Python/3.8/lib/

In [ ]:
#@title set up virtual display

from pyvirtualdisplay import Display
os.environ['DISPLAY'] = ':0'
display = Display(visible=0, size=(1400, 900))
display.start();

In [6]:
#@title test virtual display

#@markdown If you see a video of a four-legged ant fumbling about, setup is complete!

import gym
import matplotlib
matplotlib.use('Agg')
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
)
  
where_vid = os.path.expanduser('~').join('/Documents/Python/BerkeleyDRL_fall2021/hw2/content/video')
env = wrap_env(gym.make("Ant-v2"), where_vid)

observation = env.reset()
for i in range(100):
    env.render(mode='rgb_array')
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
        break;

env.close()
print('Loading video...')
show_video(where_vid)

PermissionError: [Errno 13] Permission denied: '//Users/ikasouD'

In [27]:
 os.path.dirname(os.path.abspath(os.path.curdir))

'/Users/ikasou'

## Run Policy Gradients

In [12]:
#@title imports

import os
import time

from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.pg_agent import PGAgent

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'cs285.agents.pg_agent'

In [9]:
#@title runtime arguments

class Args:

    def __getitem__(self, key):
        return getattr(self, key)

    def __setitem__(self, key, val):
        setattr(self, key, val)

    def __contains__(self, key):
        return hasattr(self, key)

    env_name = 'CartPole-v0' #@param
    exp_name = 'q1_sb_rtg_na' #@param

    #@markdown main parameters of interest
    n_iter = 100 #@param {type: "integer"}

    ## PDF will tell you how to set ep_len
    ## and discount for each environment
    ep_len = 200 #@param {type: "integer"}
    discount = 0.95 #@param {type: "number"}

    reward_to_go = True #@param {type: "boolean"}
    nn_baseline = False #@param {type: "boolean"}
    gae_lambda = None #@param {type: "number"}
    dont_standardize_advantages = False #@param {type: "boolean"}

    #@markdown batches and steps
    batch_size = 1000 #@param {type: "integer"}
    eval_batch_size = 400 #@param {type: "integer"}

    num_agent_train_steps_per_iter = 1 #@param {type: "integer"}
    learning_rate =  5e-3 #@param {type: "number"}

    #@markdown MLP parameters
    n_layers = 2 #@param {type: "integer"}
    size = 64 #@param {type: "integer"}

    #@markdown system
    save_params = False #@param {type: "boolean"}
    no_gpu = False #@param {type: "boolean"}
    which_gpu = 0 #@param {type: "integer"}
    seed = 1 #@param {type: "integer"}

    action_noise_std = 0 #@param {type: "number"}

    #@markdown logging
    ## default is to not log video so
    ## that logs are small enough to be
    ## uploaded to gradscope
    video_log_freq =  -1#@param {type: "integer"}
    scalar_log_freq =  1#@param {type: "integer"}


args = Args()

## ensure compatibility with hw1 code
args['train_batch_size'] = args['batch_size']

if args['video_log_freq'] > 0:
    import warnings
    warnings.warn(
      '''\nLogging videos will make eventfiles too'''
      '''\nlarge for the autograder. Set video_log_freq = -1'''
      '''\nfor the runs you intend to submit.''')

In [10]:
#@title create directory for logging

data_path = ex_path + '/data'

if not (os.path.exists(data_path)):
    os.makedirs(data_path)

logdir = args.exp_name + '_' + args.env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

In [11]:
## define policy gradient trainer

class PG_Trainer(object):

    def __init__(self, params):

        #####################
        ## SET AGENT PARAMS
        #####################

        computation_graph_args = {
            'n_layers': params['n_layers'],
            'size': params['size'],
            'learning_rate': params['learning_rate'],
            }

        estimate_advantage_args = {
            'gamma': params['discount'],
            'standardize_advantages': not(params['dont_standardize_advantages']),
            'reward_to_go': params['reward_to_go'],
            'nn_baseline': params['nn_baseline'],
            'gae_lambda': params['gae_lambda'],
        }

        train_args = {
            'num_agent_train_steps_per_iter': params['num_agent_train_steps_per_iter'],
        }

        agent_params = {**computation_graph_args, **estimate_advantage_args, **train_args}

        self.params = params
        self.params['agent_class'] = PGAgent
        self.params['agent_params'] = agent_params
        self.params['batch_size_initial'] = self.params['batch_size']

        ################
        ## RL TRAINER
        ################

        self.rl_trainer = RL_Trainer(self.params)

    def run_training_loop(self):

        self.rl_trainer.run_training_loop(
            self.params['n_iter'],
            collect_policy = self.rl_trainer.agent.actor,
            eval_policy = self.rl_trainer.agent.actor,
            )

In [ ]:
## run training

print(args.logdir)
trainer = PG_Trainer(args)
trainer.run_training_loop()

In [ ]:
#@markdown You can visualize your runs with tensorboard from within the notebook

## requires tensorflow==2.3.0
%load_ext tensorboard
%tensorboard --logdir /content/cs285_f2021/homework_fall2021/hw2/data